Capstone Project: The Battle of Neighbourhoods

# 1. Introduction

## 1.1 Description of the Problem

Calgary, a cosmopolitan Alberta city with numerous skyscrapers, owes its rapid growth to its status as the centre of Canada’s oil industry. However, it’s still steeped in the western culture that earned it the nickname “Cowtown,” evident in the Calgary Stampede, its massive July rodeo and festival that grew out of the farming exhibitions once presented here.


## 1.2 Discussion of the Background

My client runs a the owner of a successful restaurant chain in the Western Region of Canada with 5 restaurants spread between Ottawa, Toronto and  a Montreal now wants to explore the western restaurant market and has decided to start with the City of Calgary.

As a restaurant owner, it is important that your business is not too close to competitors while ensuring that it is also close to major attractions in the city. Therefore, my client wants to know the best are to locate the new restaurant that will be have the least competition and the most attractions.


## 1.3 Target Audience


This project is to anyone that wants to set up business in Calgary and will like to know what the competition hotspot is like around the city.



# 2. Data

This project relies on public data from Wikipedia and Foursquare.

The neighbourhoods in Calgary is gotten from the Wikipedia page "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary" which serves as the foundation for the analysis.

Data about venues in Calgary were gotten from the FourSquare API. Additional analysis was performed by my client to provide a weighted rank of how much competition or attract a venue type is on a scale of 1-10.

For example, an hotel has a higher rank of attraction (10) and a minimal rank of competition (1) giving it a total weight of 9 (10-1).

However, a Pizza Place is a direct competitor and has a lower attraction rank of (3) and a medium competition rank of (6), giving it a total weight of -3 (3-6).

The expectation is that for a cluster of venues, we take the average of all venue weights and the cluster with the highest average weight is the best location for our restaurant.


In [1]:
import pandas as pd
import numpy as np
import requests

Load wikipedia table to dataframe

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary',header=0)[0]
df = df.rename(index=str, columns = {'Name[9]':'Name', 'Sector[10]': 'Sector', 'Ward[11]': 'Ward', 'Type[10]': 'Type', '2012 PopulationRank': 'PopulationRank2012', 'Population(2012)[9]': 'Population2012', 'Population(2011)[9]': 'Population2011', '% change': 'PercentageChange', 'Dwellings(2012)[9]': 'Dwellings2012', 'Area(km2)[10]': 'Area', 'Populationdensity': 'PopulationDensity'})
print(df.shape)
df.head()

(258, 12)


,Name,Quadrant,Sector,Ward,Type,PopulationRank2012,Population2012,Population2011,PercentageChange,Dwellings2012,Area,PopulationDensity
0,Abbeydale,NE/SE,Northeast,10,Residential,82.0,5917.0,5700.0,3.8,2023.0,1.7,3480.6
1,Acadia,SE,South,9,Residential,27.0,10705.0,10615.0,0.8,5053.0,3.9,2744.9
2,Albert Park/Radisson Heights,SE,East,10,Residential,75.0,6234.0,6217.0,0.3,2709.0,2.5,2493.6
3,Altadore,SW,Centre,11,Residential,39.0,9116.0,8907.0,2.3,4486.0,2.9,3143.4
4,Alyth/Bonnybrook,SE,Centre,9,Industrial,208.0,16.0,17.0,−5.9,14.0,3.8,4.2


In [3]:
#Remove the 'Total City of Calgary' row
df = df[df['Name'] != 'Total City of Calgary']
df.head()

,Name,Quadrant,Sector,Ward,Type,PopulationRank2012,Population2012,Population2011,PercentageChange,Dwellings2012,Area,PopulationDensity
0,Abbeydale,NE/SE,Northeast,10,Residential,82.0,5917.0,5700.0,3.8,2023.0,1.7,3480.6
1,Acadia,SE,South,9,Residential,27.0,10705.0,10615.0,0.8,5053.0,3.9,2744.9
2,Albert Park/Radisson Heights,SE,East,10,Residential,75.0,6234.0,6217.0,0.3,2709.0,2.5,2493.6
3,Altadore,SW,Centre,11,Residential,39.0,9116.0,8907.0,2.3,4486.0,2.9,3143.4
4,Alyth/Bonnybrook,SE,Centre,9,Industrial,208.0,16.0,17.0,−5.9,14.0,3.8,4.2


We only need a few fields

In [4]:
df = df[['Name', 'Quadrant', 'Sector', 'Ward', 'Type', 'Area']]
df.head()

,Name,Quadrant,Sector,Ward,Type,Area
0,Abbeydale,NE/SE,Northeast,10,Residential,1.7
1,Acadia,SE,South,9,Residential,3.9
2,Albert Park/Radisson Heights,SE,East,10,Residential,2.5
3,Altadore,SW,Centre,11,Residential,2.9
4,Alyth/Bonnybrook,SE,Centre,9,Industrial,3.8


Merger Heighbourhoods with same Postcode and Borough

In [5]:
import geocoder

latlng = []

for index, row in df.iterrows():
    latlng.append(geocoder.arcgis('{}, Calgary, Canada'.format(row['Name'])).latlng)

In [6]:
df_latlng = pd.DataFrame(latlng, columns = ['Latitude', 'Longitude'])

df.index = df_latlng.index
df['Latitude'] = df_latlng['Latitude']
df['Longitude'] = df_latlng['Longitude']

df.head()

,Name,Quadrant,Sector,Ward,Type,Area,Latitude,Longitude
0,Abbeydale,NE/SE,Northeast,10,Residential,1.7,51.05976,-113.92546
1,Acadia,SE,South,9,Residential,3.9,50.97227,-114.05843
2,Albert Park/Radisson Heights,SE,East,10,Residential,2.5,51.04200,-113.99683
3,Altadore,SW,Centre,11,Residential,2.9,51.01601,-114.10558
4,Alyth/Bonnybrook,SE,Centre,9,Industrial,3.8,51.01155,-114.04481


In [118]:
# Foursquare API keys is saved as json file.

import json
filename = 'fsquare_credential.json'
with open(filename) as f:
    data = json.load(f)

CLIENT_ID = data['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = data['CLIENT_SECRET'] # your Foursquare Secret
VERSION = data['VERSION'] # Foursquare API version

LIMIT = 200
radius = 2000

'IPFICTXR4AASJIB2RGP5GPWYAMATSIS0MGHJFTOM2AL1UZQS'

In [8]:
def get_url(lat, lng):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    return url



def get_results():
    results = []
    for index, row in df.iterrows():
        url = get_url(row['Latitude'], row['Longitude'])
        results.append(requests.get(url).json())
    return results


results = get_results()

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
from pandas.io.json import json_normalize

def get_nearby_venues(result):
    try:
        venues = result['response']['groups'][0]['items']

        nearby_venues = json_normalize(venues) # flatten JSON

        # filter columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby_venues =nearby_venues.loc[:, filtered_columns]

        # filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

        # clean columns
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

        return nearby_venues
    except:
        return None

In [11]:
nearby_venues = []
for i in range(257):
    result = results[i]
    venues = get_nearby_venues(result)
    if(venues is not None):
        nearby_venues.append(venues)
#         print(len(venues))

# nearby_venues

In [12]:
all_calgary_venues = pd.concat(nearby_venues, ignore_index=True)
all_calgary_venues = all_calgary_venues.drop_duplicates()
all_calgary_venues.head()

,name,categories,lat,lng
0,Atlas Pizza and Sports Bar,Pizza Place,51.052481,-113.941859
1,A&W Canada,Fast Food Restaurant,51.068291,-113.933571
2,Costco Wholesale,Warehouse Store,51.043141,-113.917518
3,Subway,Sandwich Place,51.059215,-113.934836
4,Calgary Co-op,Grocery Store,51.068719,-113.934014


In [29]:
all_calgary_venues_unique = all_calgary_venues['categories'].value_counts().to_frame(name='Count')

In [30]:
all_calgary_venues_unique.to_csv('all_calgary_venues_categroies.csv')
all_calgary_venues_unique = pd.read_csv('all_calgary_venues_categroies_with_weights.csv')
all_calgary_venues.head()

,name,categories,lat,lng
0,Atlas Pizza and Sports Bar,Pizza Place,51.052481,-113.941859
1,A&W Canada,Fast Food Restaurant,51.068291,-113.933571
2,Costco Wholesale,Warehouse Store,51.043141,-113.917518
3,Subway,Sandwich Place,51.059215,-113.934836
4,Calgary Co-op,Grocery Store,51.068719,-113.934014


In [41]:
all_calgary_venues_unique = pd.read_csv('all_calgary_venues_categroies_with_weights.csv')
all_calgary_venues_unique = all_calgary_venues_unique[['Categories', 'Count', 'Attraction', 'Competition']]
all_calgary_venues_unique.head()

weights = []

for index, row in all_calgary_venues.iterrows():
#     cat_weight = all_calgary_venues_unique['Categories'] == row['categories']
    
    cat_weight = all_calgary_venues_unique[all_calgary_venues_unique['Categories'] == row['categories']]
    cat_weight = cat_weight[['Attraction', 'Competition']].values
    
#     print(index)
#     print(row['categories'])
    
    attr = cat_weight[0][0]
    comp = cat_weight[0][1]
    weight_ = attr - comp

    cat_weight = [attr, comp, weight_]
    weights.append(cat_weight)

weights = pd.DataFrame(weights, columns = ['Attraction', 'Competition', 'Weight'])

all_calgary_venues.index = weights.index
all_calgary_venues['Attraction'] = weights['Attraction']
all_calgary_venues['Competition'] = weights['Competition']
all_calgary_venues['Weight'] = weights['Weight']

# df.head()

all_calgary_venues.head()

,Cluster Labels,name,categories,lat,lng,Attraction,Competition,Weight
0,0,Atlas Pizza and Sports Bar,Pizza Place,51.052481,-113.941859,3,6,-3
1,0,A&W Canada,Fast Food Restaurant,51.068291,-113.933571,4,8,-4
2,0,Costco Wholesale,Warehouse Store,51.043141,-113.917518,7,2,5
3,0,Subway,Sandwich Place,51.059215,-113.934836,8,4,4
4,0,Calgary Co-op,Grocery Store,51.068719,-113.934014,6,2,4


In [110]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 20

grouped_clustering = all_calgary_venues[['lat', 'lng']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10]
kmeans.labels_
all_calgary_venues = all_calgary_venues.drop(columns=['Cluster Labels'], axis=1)
all_calgary_venues.insert(0, 'Cluster Labels', kmeans.labels_)
all_calgary_venues.head()

,Cluster Labels,name,categories,lat,lng,Attraction,Competition,Weight
0,18,Atlas Pizza and Sports Bar,Pizza Place,51.052481,-113.941859,3,6,-3
1,18,A&W Canada,Fast Food Restaurant,51.068291,-113.933571,4,8,-4
2,18,Costco Wholesale,Warehouse Store,51.043141,-113.917518,7,2,5
3,18,Subway,Sandwich Place,51.059215,-113.934836,8,4,4
4,18,Calgary Co-op,Grocery Store,51.068719,-113.934014,6,2,4


In [111]:
all_calgary_venues[all_calgary_venues['Cluster Labels'] == 1]

,Cluster Labels,name,categories,lat,lng,Attraction,Competition,Weight
2776,1,Not Just Flowers,Gift Shop,49.125371,-53.605406,7,3,4
2777,1,Shoppers Drug Mart,Pharmacy,49.118820,-53.597998,2,0,2
2778,1,Shoppers Drug Mart,Pharmacy,49.131352,-53.592514,2,0,2


In [112]:
# kcluster_sum = all_calgary_venues.groupby('Cluster Labels').sum()
kcluster_sum = all_calgary_venues.groupby('Cluster Labels').aggregate({'Weight': 'sum', 'Competition': 'count'})
kcluster_sum = kcluster_sum.rename(index=str, columns = {'Competition':'Count'})
# print(df.shape)
kcluster_sum['avg'] =  kcluster_sum['Weight'] / kcluster_sum['Count']
kcluster_sum = kcluster_sum.sort_values(by = ['avg'], ascending = [False])
kcluster_sum.head()

,Weight,Count,avg
Cluster Labels,,,
14,207,66,3.136364
13,259,85,3.047059
1,8,3,2.666667
10,497,190,2.615789
0,434,172,2.523256


The cluster with the highest average value is the best location for our restaurant.

In [113]:
best_cluster = all_calgary_venues[all_calgary_venues['Cluster Labels'] == 14]
print(best_cluster['lat'].mean())
best_cluster.head()

51.135372980416946


,Cluster Labels,name,categories,lat,lng,Attraction,Competition,Weight
1093,14,Yakima Social Kitchen + Bar,Gastropub,51.130619,-114.007792,8,4,4
1094,14,Homewood Suites by Hilton™ - Calgary-Airport,Hotel,51.142683,-114.012082,10,1,9
1095,14,Wyndham Garden Calgary Airport,Hotel,51.140607,-114.026205,10,1,9
1096,14,Starbucks,Coffee Shop,51.132087,-114.011475,6,2,4
1097,14,International Baggage Claim 1,Airport Service,51.130297,-114.004035,7,2,5


In [114]:
import folium
# calgary_latlng = [51.039558, -114.063883]
center_of_cluster = [best_cluster['lat'].mean(), best_cluster['lng'].mean()]
map = folium.Map(location=center_of_cluster, zoom_start=14)

for i in range(0,len(best_cluster)):
    folium.Marker([best_cluster.iloc[i]['lat'], best_cluster.iloc[i]['lng']], popup=best_cluster.iloc[i]['name']).add_to(map)
    
map

## The best location for the restaurant is near the airport. This is inline with what we are exepecting